# Scenario 02: AG-UI Protocol Interface

**Estimated Time**: 45 minutes

## Learning Objectives
- Understand the AG-UI streaming protocol
- Build an AG-UI compatible server with FastAPI
- Handle lifecycle and message events
- Integrate agents with frontend interfaces

## Prerequisites
- Completed Scenario 01 (Simple Agent + MCP)
- Basic understanding of Server-Sent Events (SSE)

## Part 1: Understanding AG-UI Protocol

### What is AG-UI?

AG-UI (Agent-User Interface) is a protocol for building streaming chat interfaces.
It defines how agents communicate with user interfaces in real-time.

### Key Concepts

1. **Server-Sent Events (SSE)**: One-way streaming from server to client
2. **Event Types**: Structured events for different message phases
3. **Lifecycle Events**: RUN_STARTED, RUN_FINISHED, RUN_ERROR
4. **Message Events**: TEXT_MESSAGE_START, TEXT_MESSAGE_CONTENT, TEXT_MESSAGE_END
5. **Tool Events**: TOOL_CALL_START, TOOL_CALL_ARGS, TOOL_CALL_END

### Event Flow

```
RUN_STARTED
    ↓
TEXT_MESSAGE_START
    ↓
TEXT_MESSAGE_CONTENT (repeated)
    ↓
TEXT_MESSAGE_END
    ↓
RUN_FINISHED
```

## Part 1b: SDK-Based Implementation

This notebook demonstrates AG-UI implementation using official SDK packages:

### 🟢 Packages Used
- **`ag-ui-core`**: Official AG-UI types, events, and EventEncoder
- **`agent-framework-ag-ui`**: FastAPI integration helpers

### Benefits
- ✅ Official types ensure protocol compliance
- ✅ EventEncoder handles SSE formatting correctly
- ✅ Full Pydantic model validation
- ✅ 26 event types (vs 12 minimal types)
- ✅ Production-ready client and server wrappers

## Part 2: Setting Up the Environment

In [1]:
# Verify imports
import sys
import asyncio
from pathlib import Path

# Add project root to path
project_root = Path("..").resolve()
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

# Load environment variables
from dotenv import load_dotenv
load_dotenv(project_root / ".env")

print(f"✅ Project root: {project_root}")

# Ensure we can import from src
project_root = Path.cwd()
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

# Import SDK packages
from ag_ui.core import EventType, RunStartedEvent, TextMessageContentEvent, BaseEvent
from ag_ui.encoder import EventEncoder

# Import SDK wrappers from our module
from src.agents import (
    AGUIClient,
    AGUIAgentProtocol,
    create_agui_endpoint,
)

# Import telemetry
from src.common.telemetry import setup_telemetry, get_tracer

# Setup telemetry
setup_telemetry()
tracer = get_tracer(__name__)

print("✅ AG-UI SDK components imported successfully!")
print(f"\nEventType values ({len(list(EventType))} total):")
for i, et in enumerate(EventType, 1):
    if i <= 5:
        print(f"  {i}. {et.value}")
    elif i == 6:
        print("  ...")
print(f"  (and {len(list(EventType)) - 5} more)")

✅ Project root: C:\Users\jonasrotter\OneDrive - Microsoft\Desktop\Jonas Privat\MyCodingProjects\agents-workshop
Patch applied successfully
✓ Applied clr_loader patch for .NET 10+ compatibility
[tfm] Applying fix for version: 10.0.1 -> net10.0
[floor_version] Applying fix for version: 10.0.1 -> 10.0.0
✅ AG-UI SDK components imported successfully!

EventType values (26 total):
  1. TEXT_MESSAGE_START
  2. TEXT_MESSAGE_CONTENT
  3. TEXT_MESSAGE_END
  4. TEXT_MESSAGE_CHUNK
  5. THINKING_TEXT_MESSAGE_START
  ...
  (and 21 more)


## Part 3: SDK Event Types

The `ag-ui-core` package provides 26 official event types. Let's explore them.

In [2]:
# List all SDK event types
print("SDK Event Types (ag_ui.core.EventType):")
print("-" * 50)
for i, event_type in enumerate(EventType, 1):
    print(f"  {i:2}. {event_type.value}")
print(f"\nTotal: {len(list(EventType))} event types")

SDK Event Types (ag_ui.core.EventType):
--------------------------------------------------
   1. TEXT_MESSAGE_START
   2. TEXT_MESSAGE_CONTENT
   3. TEXT_MESSAGE_END
   4. TEXT_MESSAGE_CHUNK
   5. THINKING_TEXT_MESSAGE_START
   6. THINKING_TEXT_MESSAGE_CONTENT
   7. THINKING_TEXT_MESSAGE_END
   8. TOOL_CALL_START
   9. TOOL_CALL_ARGS
  10. TOOL_CALL_END
  11. TOOL_CALL_CHUNK
  12. TOOL_CALL_RESULT
  13. THINKING_START
  14. THINKING_END
  15. STATE_SNAPSHOT
  16. STATE_DELTA
  17. MESSAGES_SNAPSHOT
  18. ACTIVITY_SNAPSHOT
  19. ACTIVITY_DELTA
  20. RAW
  21. CUSTOM
  22. RUN_STARTED
  23. RUN_FINISHED
  24. RUN_ERROR
  25. STEP_STARTED
  26. STEP_FINISHED

Total: 26 event types


In [3]:
# Using the SDK EventEncoder for proper SSE format
encoder = EventEncoder()

# Create SDK events and encode them
run_started = RunStartedEvent(thread_id="demo-thread", run_id="demo-run")
text_content = TextMessageContentEvent(message_id="msg-1", delta="Hello, ")

print("EventEncoder converts events to SSE format:")
print()

print("1. RunStartedEvent:")
encoded = encoder.encode(run_started)
print(f"   Raw: {repr(encoded)}")
print(f"   Display:\n{encoded}")

print("2. TextMessageContentEvent:")
encoded = encoder.encode(text_content)
print(f"   Raw: {repr(encoded)}")
print(f"   Display:\n{encoded}")

EventEncoder converts events to SSE format:

1. RunStartedEvent:
   Raw: 'data: {"type":"RUN_STARTED","threadId":"demo-thread","runId":"demo-run"}\n\n'
   Display:
data: {"type":"RUN_STARTED","threadId":"demo-thread","runId":"demo-run"}


2. TextMessageContentEvent:
   Raw: 'data: {"type":"TEXT_MESSAGE_CONTENT","messageId":"msg-1","delta":"Hello, "}\n\n'
   Display:
data: {"type":"TEXT_MESSAGE_CONTENT","messageId":"msg-1","delta":"Hello, "}




## Part 4: SDK EventEncoder

The `EventEncoder` class properly formats events as SSE (Server-Sent Events).
This is the standard way to serialize AG-UI events for streaming.

## Part 5: SDK Client Wrapper

The `AGUIClient` provides a convenient wrapper around `AGUIChatClient` from the SDK.

In [4]:
# AGUIClient - wrapper for SDK's AGUIChatClient
# Create a client instance (for connecting to an AG-UI server)
client = AGUIClient(
    endpoint="http://localhost:8001",
    timeout=30.0
)

print("AGUIClient Configuration:")
print(f"  endpoint: {client.endpoint}")
print(f"  timeout:  {client.timeout}s")
print(f"  type:     {type(client).__name__}")
print()
print("Usage: client can be used to send messages to an AG-UI server")
print("       and receive streaming event responses.")

AGUIClient Configuration:
  endpoint: http://localhost:8001
  timeout:  30.0s
  type:     AGUIClient

Usage: client can be used to send messages to an AG-UI server
       and receive streaming event responses.


In [5]:
# create_agui_endpoint - simplified server setup
from fastapi import FastAPI

print("create_agui_endpoint() Helper Function")
print("=" * 50)
print()
print("Usage:")
print("  app = FastAPI()")
print("  agent = MyAgent()  # Must implement AGUIAgentProtocol")
print("  create_agui_endpoint(app, agent, path='/agui')")
print()
print("This is equivalent to:")
print("  from agent_framework_ag_ui import add_agent_framework_fastapi_endpoint")
print("  add_agent_framework_fastapi_endpoint(app, agent, '/agui')")
print()
print("Benefits:")
print("  ✅ Automatic SSE streaming setup")
print("  ✅ Proper content-type headers")
print("  ✅ Works with any AGUIAgentProtocol agent")

create_agui_endpoint() Helper Function

Usage:
  app = FastAPI()
  agent = MyAgent()  # Must implement AGUIAgentProtocol
  create_agui_endpoint(app, agent, path='/agui')

This is equivalent to:
  from agent_framework_ag_ui import add_agent_framework_fastapi_endpoint
  add_agent_framework_fastapi_endpoint(app, agent, '/agui')

Benefits:
  ✅ Automatic SSE streaming setup
  ✅ Proper content-type headers
  ✅ Works with any AGUIAgentProtocol agent


In [6]:
# AGUIAgentProtocol - type checking for SDK compatibility
from typing import runtime_checkable, Protocol, AsyncIterator
from ag_ui.core import RunAgentInput

print("AGUIAgentProtocol")
print("=" * 50)
print()
print("A runtime-checkable Protocol for AG-UI compatible agents.")
print()
print("Required method signature:")
print("  async def run(self, input: RunAgentInput) -> AsyncIterator[BaseEvent]")
print()
print("Example implementation:")
print("  class MyAgent:")
print("      async def run(self, input: RunAgentInput) -> AsyncIterator[BaseEvent]:")
print("          yield RunStartedEvent(thread_id=input.thread_id, run_id='r1')")
print("          yield TextMessageContentEvent(message_id='m1', delta='Hello!')")
print("          yield RunFinishedEvent(thread_id=input.thread_id, run_id='r1')")
print()
print("Validation: isinstance(agent, AGUIAgentProtocol) → True/False")

AGUIAgentProtocol

A runtime-checkable Protocol for AG-UI compatible agents.

Required method signature:
  async def run(self, input: RunAgentInput) -> AsyncIterator[BaseEvent]

Example implementation:
  class MyAgent:
      async def run(self, input: RunAgentInput) -> AsyncIterator[BaseEvent]:
          yield RunStartedEvent(thread_id=input.thread_id, run_id='r1')
          yield TextMessageContentEvent(message_id='m1', delta='Hello!')
          yield RunFinishedEvent(thread_id=input.thread_id, run_id='r1')

Validation: isinstance(agent, AGUIAgentProtocol) → True/False


In [7]:
# Part 6: Hands-On Exercise - Event Processor

class EventProcessor:
    """Process AG-UI events for terminal display."""
    
    def __init__(self):
        self.current_content = ""
    
    def process_event(self, event_dict: dict) -> str:
        """Process an event and return formatted output."""
        event_type = event_dict.get("type")
        
        if event_type == "RUN_STARTED":
            return "🚀 Starting..."
        
        elif event_type == "TEXT_MESSAGE_START":
            self.current_content = ""
            return "\n🤖 Assistant: "
        
        elif event_type == "TEXT_MESSAGE_CONTENT":
            delta = event_dict.get("delta", "")
            self.current_content += delta
            return delta
        
        elif event_type == "TEXT_MESSAGE_END":
            return "\n"
        
        elif event_type == "TOOL_CALL_START":
            tool_name = event_dict.get("toolCallName", "unknown")
            return f"\n🔧 Calling tool: {tool_name}\n"
        
        elif event_type == "RUN_FINISHED":
            return "\n✅ Complete!"
        
        elif event_type == "RUN_ERROR":
            message = event_dict.get("message", "Unknown error")
            return f"\n❌ Error: {message}"
        
        return ""

# Test the processor with SDK event format
processor = EventProcessor()

# These events match SDK's camelCase JSON format
test_events = [
    {"type": "RUN_STARTED", "threadId": "t1", "runId": "r1"},
    {"type": "TEXT_MESSAGE_START", "messageId": "m1", "role": "assistant"},
    {"type": "TEXT_MESSAGE_CONTENT", "messageId": "m1", "delta": "Hello from "},
    {"type": "TEXT_MESSAGE_CONTENT", "messageId": "m1", "delta": "AG-UI SDK!"},
    {"type": "TEXT_MESSAGE_END", "messageId": "m1"},
    {"type": "RUN_FINISHED", "threadId": "t1", "runId": "r1"},
]

print("=== Event Processor Output ===")
for event in test_events:
    output = processor.process_event(event)
    print(output, end="", flush=True)

=== Event Processor Output ===
🚀 Starting...
🤖 Assistant: Hello from AG-UI SDK!

✅ Complete!

## Summary

In this scenario, you learned about the **AG-UI SDK** approach:

1. **Event Types**: 26 official types from `ag_ui.core.EventType`
2. **EventEncoder**: Proper SSE formatting with `ag_ui.encoder.EventEncoder`
3. **AGUIClient**: SDK wrapper for connecting to AG-UI servers
4. **create_agui_endpoint**: FastAPI integration helper
5. **AGUIAgentProtocol**: Type checking for SDK-compatible agents

## Key SDK Components

| Component | Package | Purpose |
|-----------|---------|---------|
| `EventType` | ag-ui-core | 26 event type constants |
| `EventEncoder` | ag-ui-core | SSE format encoding |
| `RunAgentInput` | ag-ui-core | Request model |
| `BaseEvent` | ag-ui-core | Event base class |
| `AGUIChatClient` | agent-framework-ag-ui | HTTP client |
| `add_agent_framework_fastapi_endpoint` | agent-framework-ag-ui | Server setup |

## Next Steps

- **Scenario 3**: A2A Protocol for agent-to-agent communication
- Build a full web UI consuming AG-UI events
- Implement an AGUIAgentProtocol-compatible agent

## Resources

- [ag-ui-core Package](https://pypi.org/project/ag-ui-core/)
- [agent-framework-ag-ui Package](https://pypi.org/project/agent-framework-ag-ui/)
- [Server-Sent Events (MDN)](https://developer.mozilla.org/en-US/docs/Web/API/Server-sent_events)
- [AG-UI Protocol Spec](specs/001-agentic-patterns-workshop/contracts/agui-events.md)